In [155]:
import pandas as pd
import numpy as np

In [77]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [129]:
df = pd.read_csv('/content/final_data_kw - final_data_kw.csv', usecols= ['Description','Interests'])

In [130]:
df.Interests = df.Interests.str.rstrip("\r")
df.Interests = df.Interests.str.rstrip("\n")
df.Interests = df.Interests.str.rstrip("\r\n")

In [131]:
df.head()

,Description,Interests
0,Create a high-quality first person shooter gam...,Others
1,Machine learning is transforming the world aro...,Machine Learning
2,"KM3NeT, a new generation of neutrino telescope...",Science
3,A novel approach for the fusion of detection s...,Science
4,This book shows JavaScript developers how to b...,Web Dev


In [132]:
df = df[:250]

In [133]:
df.isnull().sum()

Description    0
Interests      0
dtype: int64

In [134]:
df_train, df_test = train_test_split(df, test_size=0.2)

In [135]:
df_train.Interests.nunique()

13

In [136]:
encoded_dict = {'Ethics':0,'Machine Learning':1,'Deep Learning':2,
                'Artificial Intelligence':3,'DSA':4,'Business':5,
                'Science':6,'Cryptography':7,'CS Fundamentals':8,
                'Web Dev':9,'App Dev':10,'Technology':11,'Others':12}

In [137]:
df_train['Interests'] = df_train.Interests.map(encoded_dict)
df_test['Interests'] = df_test.Interests.map(encoded_dict)

In [138]:
df_test.head()

,Description,Interests
132,Intercept problems and challenges typically fa...,8
168,"Real-world examples of cross-browser, mobile, ...",10
69,Enter the era of medical research using mobile...,11
207,"When, in 1984–86, Richard P. Feynman gave his ...",11
78,We construct compositional continuous approxim...,6


In [105]:
ohe = OneHotEncoder()

In [74]:
y_train = ohe.fit([df_train.Interests])
y_test = ohe.fit([df_test.Interests])

In [139]:
y_train = to_categorical(df_train.Interests)
y_test = to_categorical(df_test.Interests)

In [140]:
df_train.Interests.unique()

array([ 2, 11, 10,  6,  9,  8,  4,  1,  0,  3, 12,  5,  7])

In [141]:
df_test.Interests.unique()

array([ 8, 10, 11,  6,  1,  5,  9,  0,  2, 12,  4,  7,  3])

In [142]:
y_train

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [143]:
!pip install transformers
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [144]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [145]:
x_train = tokenizer(
    text = df_train.Description.tolist(),
    add_special_tokens = True,
    max_length = 100,
    truncation = True,
    padding = True,
    return_tensors = 'tf',
    return_attention_mask = True,
    verbose = True
)

x_test = tokenizer(
    text=df_test.Description.tolist(),
    add_special_tokens=True,
    max_length=100,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_attention_mask = True,
    verbose = True
)

In [146]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [147]:
x_train['input_ids']

<tf.Tensor: shape=(200, 100), dtype=int32, numpy=
array([[  101,  1457,  8741, ..., 26463,  1110,   102],
       [  101, 12958,  4558, ...,     0,     0,     0],
       [  101,   140, 15998, ...,     0,     0,     0],
       ...,
       [  101,  1799,  9066, ...,  1116,  1169,   102],
       [  101, 12958,  4558, ...,     0,     0,     0],
       [  101,  1284,  1675, ...,  1162,  2064,   102]], dtype=int32)>

In [148]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [149]:
max_len = 100
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(64, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(16,activation = 'relu')(out)
y = Dense(13,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [150]:
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [152]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=10,
    batch_size=8
)

Epoch 1/10
25/25 [==============================] - 6s 222ms/step - loss: 1.5671 - balanced_accuracy: 0.5750 - val_loss: 2.0935 - val_balanced_accuracy: 0.3200
Epoch 2/10
25/25 [==============================] - 6s 224ms/step - loss: 1.2480 - balanced_accuracy: 0.6950 - val_loss: 2.1031 - val_balanced_accuracy: 0.3200
Epoch 3/10
25/25 [==============================] - 5s 213ms/step - loss: 1.0430 - balanced_accuracy: 0.7200 - val_loss: 2.0781 - val_balanced_accuracy: 0.3600
Epoch 4/10
25/25 [==============================] - 5s 212ms/step - loss: 0.8242 - balanced_accuracy: 0.8150 - val_loss: 2.0509 - val_balanced_accuracy: 0.4000
Epoch 5/10
25/25 [==============================] - 5s 214ms/step - loss: 0.6231 - balanced_accuracy: 0.8800 - val_loss: 1.9448 - val_balanced_accuracy: 0.4000
Epoch 6/10
25/25 [==============================] - 5s 216ms/step - loss: 0.4939 - balanced_accuracy: 0.8950 - val_loss: 1.9369 - val_balanced_accuracy: 0.4200
Epoch 7/10
25/25 [======================

In [153]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})

In [157]:
y_predicted = np.argmax(predicted_raw, axis = 1)
y_true = df_test.Interests

In [158]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       1.00      0.40      0.57         5
           2       0.60      0.60      0.60         5
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         2
           5       0.50      1.00      0.67         2
           6       0.62      0.83      0.71         6
           7       0.00      0.00      0.00         1
           8       0.11      0.14      0.12         7
           9       0.57      0.57      0.57         7
          10       1.00      0.25      0.40         4
          11       0.38      0.43      0.40         7
          12       0.00      0.00      0.00         1

    accuracy                           0.42        50
   macro avg       0.37      0.33      0.31        50
weighted avg       0.48      0.42      0.42        50



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [159]:
df['Description'][124]

"This course is intended to be an introduction to machine learning for non-technical business professionals. There is a lot of hype around machine learning and many people are concerned that in order to use machine learning in business, you need to have a technical background. For reasons that are covered in this course, that's not the case. In actuality, your knowledge of your business is far more important than your ability to build an ML model from scratch. By the end of this course, you will have learned how to: � Formulate machine learning solutions to real-world problems � Identify whether the data you have is sufficient for ML � Carry a project through various ML phases including training, evaluation, and deployment � Perform AI responsibly and avoid reinforcing existing bias � Discover ML use cases � Be successful at ML You'll need a desktop web browser to run this course's interactive labs via Qwiklabs and Google Cloud Platform. >>> By enrolling in this course you agree to the

In [161]:
texts = """
This course is intended to be an introduction to machine learning for non-technical business professionals. There is a lot of hype around machine learning and many people are concerned that in order to use machine learning in business, you need to have a technical background. For reasons that are covered in this course, that's not the case. In actuality, your knowledge of your business is far more important than your ability to build an ML model from scratch. By the end of this course, you will have learned how to: � Formulate machine learning solutions to real-world problems � Identify whether the data you have is sufficient for ML � Carry a project through various ML phases including training, evaluation, and deployment � Perform AI responsibly and avoid reinforcing existing bias � Discover ML use cases � Be successful at ML You'll need a desktop web browser to run this course's interactive labs via Qwiklabs and Google Cloud Platform. >>> By enrolling in this course you agree to the Qwiklabs Terms of Service as set out in the FAQ and located at: https://qwiklabs.com/terms_of_service
"""
x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=100,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True) 
validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
for key , value in zip(encoded_dict.keys(),validation[0]):
    print(key,value)

Ethics 98.23955
Machine Learning 99.88696
Deep Learning 93.39481
Artificial Intelligence 6.037698
DSA 87.975815
Business 81.39188
Science 44.214516
Cryptography 91.974815
CS Fundamentals 70.83619
Web Dev 82.95198
App Dev 7.2468452
Technology 88.651215
Others 1.5485514


In [162]:
model.save('my_model.h5')

In [165]:
new_model = tf.keras.models.load_model('/content/my_model.h5', custom_objects={'TFBertModel': TFBertModel})

# Show the model architecture
new_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 tf_bert_model_2 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 100,                                         

In [166]:
texts = """
This course is intended to be an introduction to machine learning for non-technical business professionals. There is a lot of hype around machine learning and many people are concerned that in order to use machine learning in business, you need to have a technical background. For reasons that are covered in this course, that's not the case. In actuality, your knowledge of your business is far more important than your ability to build an ML model from scratch. By the end of this course, you will have learned how to: � Formulate machine learning solutions to real-world problems � Identify whether the data you have is sufficient for ML � Carry a project through various ML phases including training, evaluation, and deployment � Perform AI responsibly and avoid reinforcing existing bias � Discover ML use cases � Be successful at ML You'll need a desktop web browser to run this course's interactive labs via Qwiklabs and Google Cloud Platform. >>> By enrolling in this course you agree to the Qwiklabs Terms of Service as set out in the FAQ and located at: https://qwiklabs.com/terms_of_service
"""
x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=100,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True) 
validation = new_model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
for key , value in zip(encoded_dict.keys(),validation[0]):
    print(key,value)

Ethics 98.23955
Machine Learning 99.88696
Deep Learning 93.39481
Artificial Intelligence 6.037698
DSA 87.975815
Business 81.39188
Science 44.214516
Cryptography 91.974815
CS Fundamentals 70.83619
Web Dev 82.95198
App Dev 7.2468452
Technology 88.651215
Others 1.5485514
